# Model Selection {#sec-model-selection}

In [Chapter -@sec-linear-regression] and [Chapter -@sec-logistic-regression], we included one simple method for model selection, stepwise selection. This chapter expands upon our model selection tools in R by focusing on regularized regression\index{regularized regression}. The two packages we cover are **glmnet** [@R-glmnet] \index{R packages!glmnet} and **L0Learn** [@R-L0Learn]\index{R packages!L0Learn}. These two packages focus on different types of model regularization.

In [ ]:
#| warning: false

library(HDSinRdata)
library(tidyverse)
library(glmnet)
library(L0Learn)

To demonstrate these packages, we use the same motivating example as in [Chapter -@sec-linear-regression]. Recall, that the `NHANESsample` dataset \index{Datasets!NHANESSample@\texttt{NHANESSample}} contains lead, blood pressure, BMI, smoking status, alcohol use, and demographic variables from NHANES 1999-2018. Our focus is looking at the association between blood lead levels and systolic blood pressure. We first create a single systolic blood pressure by averaging across all measurements. We also transform lead with a log transformation before dropping variables we want to exclude from our analysis.

In [ ]:
# load in data
data(NHANESsample)

# transform SBP and lead
NHANESsample$SBP <- 
  rowMeans(NHANESsample[c("SBP1", "SBP2", "SBP3", "SBP4")], 
                             na.rm=TRUE)
NHANESsample$LEAD <- log(NHANESsample$LEAD)

# remove variables not to include in the model
nhanes <- NHANESsample %>%
  select(-c(ID, HYP, LEAD_QUANTILE, DBP1, DBP2, DBP3, DBP4,
            SBP1, SBP2, SBP3, SBP4, YEAR)) %>%
  na.omit()

# convert to factors
nhanes$SEX <- factor(nhanes$SEX)
nhanes$RACE <- factor(nhanes$RACE)
nhanes$EDUCATION <- factor(nhanes$EDUCATION)
nhanes$BMI_CAT <- factor(nhanes$BMI_CAT)
nhanes$ALC <- factor(nhanes$ALC)

## Regularized Regression

Suppose we have a numeric data matrix $X \in \mathbb{R}^{n \times p}$ and outcome vector $y \in \mathbb{R}^n$. We let $x_i$ denote the vector representing the $i$th row of $X$. This corresponds to the $i$th observation. When we refer to regularized regression, we are referring to solving the following optimization problem that minimizes the average loss plus a penalty term.

$$ 
\min_{ (\beta_0, \beta) \in \mathbb{R}^{p+1}} \frac{1}{n} \sum_{i=1}^n l(y_i, \beta_0 + \beta^T x_i) + \text{Pen}(\beta)
$$ {#eq-reguarlization}

The function $l(y_i, \beta_0 + \beta^T x_i)$ represents the loss function\index{loss function}. For linear regression, this corresponds to the squared error $(y_i - \beta_0 - \beta^T x_i)^2$. For logistic regression, this loss corresponds to the logistic loss function.

The penalty terms \index{regularized regression!penalty terms} we implement include the following:

-   L0 Norm: $||\beta ||_0 = \sum_{j=1}^p 1(\beta_j \neq 0)$, the number of non-zero coefficients,

-   L1 Norm: $||\beta ||_1 = \sum_{j=1}^p |\beta_j|$, the sum of absolute values of the coefficients, and

-   Squared L2 Norm: $||\beta ||_2^2 = \sum_{j=1}^p \beta_j^2$, the sum of squared coefficients.

## Elastic Net \index{elastic net}

We first consider L1 and L2 regularization. In particular, consider the following penalty term, referred to as elastic net regularization, 

$$ 
\lambda \left[ \alpha ||\beta||_1 + (1-\alpha) ||\beta||^2_2 \right],
$$ 

where $\lambda$ is a complexity parameter and $\alpha$ controls the balance between the two norms. A model with only L1 regularization ($\alpha = 1$) corresponds to lasso regression \index{lasso regression} while a model with only L2 regularization ($\alpha=0$) corresponds to ridge regression\index{ridge regression}. Note that the penalty depends on the scale of $X$ and we typically assume each column has been standardized.

The **glmnet** package \index{R packages!glmnet} implements elastic net regularization. It assumes our data are in the form described previously. Therefore, we first create our numeric data matrix `x` and output vector `y`. Some of our variables are categorical, so in order to create a numeric matrix we need to one-hot encode them. We can do so using the `model.matrix()`\index{R functions!model.matrix()@\texttt{model.matrix()}} function which takes in a formula and a data frame and creates the corresponding design matrix including creating dummy variables from factor variables and implementing any transformations. Note that we drop the first generated column which corresponds to the intercept. The transformation to our outcome does not impact the result.

In [ ]:
x <- model.matrix(log(SBP) ~ ., nhanes)[, -1]
head(x)

Our outcome vector corresponds to log transformed systolic blood pressure.

In [ ]:
y <- log(nhanes$SBP)

The `glmnet()`\index{R functions!glmnet()@\texttt{glmnet()}} function fits an elastic net regression model. This requires us to specify our input matrix `x` and response variable `y`. Additionally, we can specify the assumed distribution for `y` using the `family` argument. In our subsequent example, we fit this model with $\alpha = 1$ and 25 different values of $\lambda$. By default, `glmnet()` sets $\alpha$ to 1 and creates a grid of 100 different values of `lambda`. It is also the default to standardize `x`, which we can turn off by specifying `standardize = FALSE` in our function call.

In [ ]:
mod_lasso <- glmnet(x, y, family = "gaussian", alpha = 1,
                    nlambda = 25)

If we plot the resulting object, we can see the model coefficients for each resulting model by plotting how the coefficient for each variable changes with the value of $\lambda$. The `plot()` \index{R functions!plot()@\texttt{plot()}} function by default plots these against the penalty term but we can also specify to plot against the $\lambda$ values on the log scale. The `label` argument adds a label to each line, though these are often hard to read. The numbers at the top of the plot indicate how many non-zero coefficients were included in the model for different $\lambda$ values. Read the documentation `?glmnet` to see the other possible inputs including the `penalty.factor` and `weights` arguments.

In [ ]:
plot(mod_lasso, xvar = "lambda", label = TRUE)

We can also print our results. This prints a matrix with the values of $\lambda$ used. For each $\lambda$ value we can also see the number of non-zero coefficients (`Df`) and the percent deviance explained (`%dev`).

In [ ]:
print(mod_lasso)

We can extract the model for a particular value of $\lambda$ using the `coef()` function\index{R functions!coef()@\texttt{coef()}}. The argument `s` specifies the value of $\lambda$. For the particular value of $\lambda$ chosen in the following code, only age has a non-zero coefficient. Note that the `coef()` function returns the coefficients on the original scale.

In [ ]:
coef(mod_lasso, s = 0.045920)

We can also use the `predict()`\index{R functions!predict()@\texttt{predict()}} function to predict blood pressure for this particular model. In the function call, we have specified our value of $\lambda$ as well as our data matrix `x` as the data to predict on.

In [ ]:
pred_lasso <- predict(mod_lasso, s = 0.045920, newx = x)

This shows our observed model fit for a fairly high penalty term. In order to choose the best value of $\lambda$, we use 5-fold cross-validation. First, we randomly assign each observation to one of five folds using the `sample()`\index{R functions!sample()@\texttt{sample()}} function. We can see that this splits the data into folds of roughly equal size.

In [ ]:
set.seed(1)
foldid <- sample(1:5, size = nrow(x), replace = TRUE)
table(foldid)

Next, we call the `cv.glmnet()`\index{R functions!cv.glmnet()@\texttt{cv.glmnet()}} function which implements k-fold cross-validation \index{cross-validation} across a grid of $\lambda$ values. Similar to before, we specified `x`, `y`, and `alpha = 1`. This time, we also include the measure to use for cross-validation (`mse` indicates mean squared error) and provide the fold vector `foldid`. If you do not want to provide folds, you can instead use the `nfolds` argument to specify the number of folds desired and the `cv.glmnet()` function will create them. Plotting the returned object shows us the estimated mean squared error for different values of $\lambda$ as well as error bars for each estimate. Similar to before, it also shows the number of non-zero coefficients at the top.

In [ ]:
cv_lasso <- cv.glmnet(x, y, foldid = foldid, alpha = 1,
                      type.measure = "mse")
plot(cv_lasso)

There are two vertical dashed lines included in the plot. These correspond to two values of $\lambda$ that are stored in our object. The first is `lambda.min`. This corresponds to the $\lambda$ value with the lowest estimated mean squared error. The other is `lambda.1se`. This corresponds to the largest $\lambda$ value whose estimated mean squared error is within one standard error of the lowest value. As indicated in the plot, this corresponds to a model with fewer included coefficients and higher regularization.

We use the `lambda.min` value as our chosen $\lambda$ value. To extract the coefficients corresponding to this $\lambda$ value we again use the `coef()` function. However, this $\lambda$ might not be one of the initial 25 used for our `mod_lasso` object. In this case, the `coef()` function uses linear interpolation to get the estimated coefficients. If we want to refit our model for this particular value of $\lambda$, we can instead specify the argument `exact = TRUE` and provide `x` and `y`.

In [ ]:
lasso_coef <- coef(mod_lasso, s = cv_lasso$lambda.min,
                   exact = TRUE, x = x, y = y)
lasso_coef

We now repeat the same process for a model with $\alpha = 0$ and $\alpha = 0.5$. In this case, we call the `glmnet()` function with our chosen $\lambda$ value to find the coefficients. This is equivalent to what we did for our lasso model. Last, we create a data frame with the estimated coefficients for each model. The `coef()` function returns a matrix, so this requires converting these to numeric vectors.

In [ ]:
# cross-validation using same folds
cv_ridge <- cv.glmnet(x, y, foldid = foldid, alpha = 0,
                      type.measure = "mse")
cv_elastic <- cv.glmnet(x, y, foldid = foldid, alpha = 0.5,
                      type.measure = "mse")

# Refit model on full data with chosen lambda
mod_ridge <- glmnet(x, y, alpha = 0, lambda = cv_ridge$lambda.min)
mod_elastic <- glmnet(x, y, alpha = 0.5, lambda = cv_ridge$lambda.min)

# extract coefficients for a table
res_coef <- data.frame(name = c("Intercept", colnames(x)),
                       lasso = round(as.numeric(lasso_coef), 3),
                       ridge = round(as.numeric(coef(mod_ridge)), 3),
                       elastic = round(as.numeric(coef(mod_elastic)), 
                                       3))
res_coef

The coefficients between the models are not so different. We can also compare their mean squared errors, which are also similar. Since our lasso model was fit on a grid of $\lambda$ values, we again have to specify this value.

In [ ]:
mean((nhanes$SBP - exp(predict(mod_lasso, newx = x,
                               s = cv_lasso$lambda.min)))^2)
mean((nhanes$SBP - exp(predict(mod_ridge, newx = x)))^2)
mean((nhanes$SBP - exp(predict(mod_elastic, newx = x)))^2)

## Best Subset \index{best subset}

The second package we introduce in this chapter is one that allows us to fit models with an L0 penalty term. The package **L0Learn** \index{R packages!L0Learn} considers penalties of the following forms.

-   L0 only: $\lambda ||\beta||_0$

-   L0L1: $\lambda ||\beta ||_0 + \gamma ||\beta||_1$

-   L0L2: $\lambda ||\beta||_0 + \gamma ||\beta ||_2^2$

To fit a model with an L0 penalty term, we use the `L0Learn.fit()`\index{R functions!L0Learn.fit()@\texttt{L0Learn.fit()}} function. Similar to `glmnet()`, we need to specify our input matrix `x` and response vector `y` as well as our penalty using the `penalty` argument. We can also specify a number of $\lambda$ values to consider `nLambda` and specify the family through the loss function `loss`. 

In [ ]:
mod_l0 <- L0Learn.fit(x, y, penalty = "L0", loss = "SquaredError",
                      nLambda = 20)

Plotting the returned object also shows how the coefficients for each variable change with the penalty term, given by the support size or number of non-zero coefficients in this case. We can also print the returned object to see the different values of $\lambda$ used and the corresponding number of included variables. 

In [ ]:
plot(mod_l0)

In [ ]:
print(mod_l0)

To extract the model for a particular value, we can use the `coef()`\index{R functions!coef()@\texttt{coef()}} function and specify the $\lambda$ and $\gamma$ values to use. 

In [ ]:
coef_l0 <- coef(mod_l0, lambda = 1.75475e-03, gamma = 0)

Unfortunately, this output doesn't include variable names so we add them manually.

In [ ]:
rownames(coef_l0) <- c("Intercept", colnames(x))
coef_l0

If instead we want to include a penalty with an L0 and L2 norm term, we can change our penalty argument to `penalty = L0L2` and specify a number of $\gamma$ values to test. 

In [ ]:
mod_l0l2 <- L0Learn.fit(x, y, penalty = "L0L2", 
                        loss = "SquaredError",
                        nLambda = 20, nGamma = 10)

The **L0Learn** package also includes a function to use cross-validation \index{cross-validation}\index{R functions!L0Learn.cvfit()@\texttt{L0Learn.cvfit()}} to choose these parameters. Unfortunately, it does not include an option to specify your own folds. Instead, we use the `nFolds` argument to specify to run 5-fold cross-validation. This function also allows us to specify a number of $\lambda$ and $\gamma$ values, or we can use the default number. Plotting the result shows the estimated mean squared error with error bars for each model and the corresponding support size. 

In [ ]:
cv_l0l2 = L0Learn.cvfit(x, y, loss = "SquaredError",
                        nFolds = 5, penalty = "L0L2")
plot(cv_l0l2)

The returned results are stored in `cvMeans`. This is a list of matrices — one for each value of $\gamma$. To extract these into a more manageable form, we use the `sapply()` function to convert each matrix to a numeric vector and create a matrix of results. The columns of this matrix correspond to the 10 $\gamma$ values used, and each column of this matrix corresponds to 100 $\lambda$ values chosen for that particular value of $\gamma$. We use the `which()` function to find which one has the lowest estimated mean squared error.

In [ ]:
cv_res <- sapply(cv_l0l2$cvMeans, as.numeric)
min_ind <- which(cv_res == min(cv_res), arr.ind = TRUE)
min_ind

We can then extract out the corresponding $\gamma$ and $\lambda$ values through the `fit` object returned in our result. 

In [ ]:
gamma_min <- cv_l0l2$fit$gamma[[min_ind[2]]]
lambda_min <- cv_l0l2$fit$lambda[[min_ind[2]]][min_ind[1]]

We find the estimated coefficients for this model using the `coef()` function on the cross-validation object `cv_l0l2`. 

In [ ]:
cv_coef_l0 <- coef(cv_l0l2, gamma = gamma_min, lambda = lambda_min)
rownames(cv_coef_l0) <- c("Intercept", colnames(x))
cv_coef_l0

Last, we use the `predict()`\index{R functions!predict()@\texttt{predict()}} function on `cv_l0l2` to evaluate our resulting model. To do so, we need to specify $\lambda$ and $\gamma$ as well as our data `x`. This model is much sparser than the ones in the previous section, but our mean squared error on our training data is a little higher.

In [ ]:
pred_l0l2 <- predict(cv_l0l2, gamma = gamma_min, 
                     lambda = lambda_min, newx = x)
mean((nhanes$SBP - exp(pred_l0l2))^2)

## Exercises

For these exercises, we use the `nyts` data from [Chapter -@sec-logistic-regression]. Our outcome of interest is whether or not someone uses any tobacco product.

1.  Create a new variable `tobacco_use` representing any tobacco use in the past 30 days (including e-cigarettes, cigarettes, and/or cigars). Then, create a new data frame `nyts_sub` that contains this new column as well as columns for location, age, sex, race, whether someone identifies with the LGBT community, grades in the past year, perceived_cigarette use, perceived e-cigarette use, psych distress, and family affluence.

2.  Create an outcome vector `y` corresponding to the column `tobacco_use` and a model matrix `x` containing all other covariates.

3.  Fit a L1 (lasso), L2 (ridge), and L0 (best subset) penalized regression model using 5-fold cross-validation. Create a data frame with the corresponding coefficients for all models. Be sure to update the loss function to reflect our binary outcome.

4.  Report the AUC and accuracy of these three models on the training data.